# Analysis based on EpiCure outputs: analysis of extruding cells compared to their neighbors

Based on the .csv files that can be generated by EpiCure, this notebook analyses the evolution of cell area of extruding cells compared to their neighbors.

*This notebook is part of EpiCure release, see https://gitlab.pasteur.fr/gletort/epicure for more informations*

In [ ]:
import os
import numpy as np
import pandas as pand

### Set the parameters to define the path where the output of EpiCure are:
- `data_dir` : main directory where the input image is
- `filename` : name of the input image, whithout the ".tif" extension
- `epic_folder`: directory where the output of EpiCure are, usually `epics`

In [ ]:
## path to input folder where EpiCured output have been saved
data_dir = os.path.join("..", "data", "selection", "t000-177")
filename = "Ecad"
epic_folder = "epics_uncorrected"

### Reads the events file that contains the list of events and their position
This file is generated by EpiCure Output `export/measure events` option.

To generate it, in EpiCure:

- Go to `Output` panel and select `Measure cell features` option
- Select `All cells`.  
- In `choose events`, select at least `extrusion`
- Choose `Csv file` for the output format of the export of events
- Click on `Export events as` to launch the export.

The file called `filename_events.csv` should be created in your `epic_folder`

In [ ]:
## Read the measured events file
events_file = os.path.join( data_dir, epic_folder, filename+"_events.csv" )
events_data = pand.read_csv( events_file, delimiter='\t' )
print(events_data)

### Reads the measures file that contains the cell measured area and their label
This file is generated by EpiCure Output `Measure cell features` option.

To generate it, in EpiCure:

- Go to `Output` panel and select `Measure cell features` option
- Select `All cells`. 
- In `choose features`, select at least `Neighbors` and `area` 
- Choose `Create feature table` to launch the measurement of all cells (it can take long for big image). 
- When it is finished, a table with the measured features will be visible in your right panel interface in EpiCure.
- Click on `Save features table` to save the results in a xlsx file.

The file called `filename_features.xlsx` should be created in your `epic_folder`

In [ ]:
## Read the measured cells file
cells_file = os.path.join( data_dir, epic_folder, filename+"_features.xlsx" )
cells_data = pand.read_excel( cells_file )
print( cells_data )

## Measure the relative area of extruding cells

For each extruding cells (linked to an extrusion events from EpiCure outputs), measure its area divided by the mean area of its neighbors at each time before extrusion

The neighbors of the cell are defined by the `Neighbor` feature saved in the measure file `filename_features.xlsx` generated by EpiCure.

In [ ]:
## get area/mean(area_neighbors) evolution in time, with tfinal=time(extrusion)
resdf = pand.DataFrame( columns=["TimeToExtrusion", "Label", "NbNeighbors", "RelativeArea"] ) 
extruding = events_data.loc[ events_data["EventClass"]=="extrusion", ["label", "frame"] ]

i = 0
for ind, label in enumerate(extruding["label"]):
    frame = extruding["frame"].iloc[ind]
    prev_frame = frame
    time = []
    rel_area = []
    while (prev_frame >= 0) and ((cells_data["label"]==label) & (cells_data["frame"]==prev_frame)).any():
        row = ((cells_data["label"]==label) & (cells_data["frame"]==prev_frame))
        area = cells_data.loc[ row, "area" ]
        neighbors = cells_data.loc[ row, "Neighbors" ]
        nei_labels = list(map(int, neighbors.iloc[0].split("&")) )
        nei_rows = ( (cells_data["label"].isin(nei_labels)) & (cells_data["frame"]==prev_frame) )
        neighbors_area = cells_data.loc[ nei_rows, "area" ].mean()
        resdf.loc[i] = [ prev_frame - frame, label, len(nei_labels), float(area)/float(neighbors_area) ]
        i = i + 1
        prev_frame = prev_frame - 1
        

In [ ]:
resdf

### Save the calculated data to an excel file

In [ ]:
 resdf.to_csv( os.path.join( data_dir, epic_folder, filename+"_extrusion_relativearea.csv" ), sep="\t")

### Plot the relative area in time, averaged over all labels

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

dfmean = resdf.groupby(['TimeToExtrusion'])['RelativeArea'].mean().reset_index()
dfsd = resdf.groupby(['TimeToExtrusion'])['RelativeArea'].std().reset_index()
dfsd["StdInf"] = dfmean["RelativeArea"] - dfsd["RelativeArea"]
dfsd["StdSup"] = dfmean["RelativeArea"] + dfsd["RelativeArea"]

figcorr = px.line( dfmean, x='TimeToExtrusion', y='RelativeArea' )
figcorr.update_traces(line=dict(color='#5F8FD9'))
figcorrsd = px.line( dfsd, x='TimeToExtrusion', y='StdInf')
figcorrsd.update_traces( line=dict(color="#5F8FD9", width=1, dash="dash") )
figcorrsdsup = px.line( dfsd, x='TimeToExtrusion', y='StdSup')
figcorrsdsup.update_traces( line=dict(color="#5F8FD9", width=1, dash="dash") )

fig = go.Figure( figcorr.data figcorrsd.data + figcorrsdsup.data  )
fig